## 행정동 별 개인형 이동장치 주차장 가중치 구하기

In [3]:
import pandas as pd

df = pd.read_csv("./data/충청남도 천안시_개인형 이동장치(PM) 주차장 현황_20240529.csv", encoding='cp949')
df.head()

,연번,주차장 위치,주차장 주소,법정동,설치일자,데이터기준일자
0,1,한양수자인 하늘약국 앞,천안시 동남구 청당동 951,청당동,2021년,2024-05-29
1,2,청수 한양수자인 베스킨라빈스 앞,천안시 동남구 청당동 951,청당동,2021년,2024-05-29
2,3,신방한라비발디 후문 정류장,천안시 동남구 신방동 1891,신방동,2021년,2024-05-29
3,4,신방우체국 중앙공원,천안시 동남구 신방동 1894,신방동,2021년,2024-05-29
4,5,통정 상쾌한 내과 앞,천안시 동남구 청당동 396-10,청당동,2021년,2024-05-29


In [5]:
from PyKakao import Local
import requests

def addr_to_lat_lon(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + ""} # 카카오 api 작성
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address']
    return float(match_first['x']), float(match_first['y'])

def add_lat_lon_columns(df):
    longitudes = [] #경도
    latitudes = [] #위도
    for address in df['주차장 주소']:
        try:
            lon,lat = addr_to_lat_lon(address)
            longitudes.append(lon)
            latitudes.append(lat)
        except Exception as e:
            print(f"Error processing address '{address}': {e}")
            latitudes.append(None)
            longitudes.append(None)
    df['Longitude'] = longitudes
    df['Latitude'] = latitudes
    return df


new_df = add_lat_lon_columns(df)
new_df

Error processing address '천안시 서북구 백석동 658-77': list index out of range
Error processing address '천안시 서북구 두정동 71-1,2': list index out of range
Error processing address '천안시 서북구 쌍용동 175': list index out of range
Error processing address '천안시 동남구 안서동 368-18': list index out of range


,연번,주차장 위치,주차장 주소,법정동,설치일자,데이터기준일자,Longitude,Latitude
0,1,한양수자인 하늘약국 앞,천안시 동남구 청당동 951,청당동,2021년,2024-05-29,127.152010,36.784222
1,2,청수 한양수자인 베스킨라빈스 앞,천안시 동남구 청당동 951,청당동,2021년,2024-05-29,127.152010,36.784222
2,3,신방한라비발디 후문 정류장,천안시 동남구 신방동 1891,신방동,2021년,2024-05-29,127.134893,36.770837
3,4,신방우체국 중앙공원,천안시 동남구 신방동 1894,신방동,2021년,2024-05-29,127.133687,36.772472
4,5,통정 상쾌한 내과 앞,천안시 동남구 청당동 396-10,청당동,2021년,2024-05-29,127.142137,36.780822
...,...,...,...,...,...,...,...,...
270,271,KG모빌리티 맞은편,천안시 서북구 쌍용동 440-2,쌍용동,2024년,2024-05-29,127.120426,36.791012
271,272,충무로,천안시 서북구 쌍용동 440-25,쌍용동,2024년,2024-05-29,127.121019,36.791838
272,273,49소갈비살/현대자동차 앞,천안시 서북구 쌍용동 450-2,쌍용동,2024년,2024-05-29,127.118048,36.790761
273,274,광혜당 약국 앞 공중전화 부스 옆,천안시 서북구 쌍용동 824,쌍용동,2024년,2024-05-29,127.132424,36.807832


In [7]:
new_df.to_csv("./data/pm_ll.csv", index=False)

In [11]:
# my_df = pd.read_csv("./data/pm.csv")
my_df = new_df

In [13]:
def get_address(address):
    url = f'https://dapi.kakao.com/v2/local/search/address.json?query={address}'
    name = ''
    headers = {"Authorization": "KakaoAK " + ""} # 카카오 api 작성
    try:
        name = requests.get(url, headers=headers).json()['documents'][0]['address']['region_3depth_h_name']
    except Exception as e:
        print(e)
        print(address)
    return name

In [15]:
def add_dong_name(df):
    dong_name = []
    for address in df['주차장 주소']:
        try:
            dong = get_address(address)
            dong_name.append(dong)
        except Exception as e:
            print(f"Error processing address '{address}': {e}")
            dong_name.append(None)
    df['법정동2'] = dong_name
    return df

my_df2 = add_dong_name(my_df)
my_df2
# range에서 벗어나는 것들은 직접 처리 

list index out of range
천안시 서북구 백석동 658-77
list index out of range
천안시 서북구 두정동 71-1,2
list index out of range
천안시 서북구 쌍용동 175
list index out of range
천안시 동남구 안서동 368-18


,연번,주차장 위치,주차장 주소,법정동,설치일자,데이터기준일자,Longitude,Latitude,법정동2
0,1,한양수자인 하늘약국 앞,천안시 동남구 청당동 951,청당동,2021년,2024-05-29,127.152010,36.784222,청룡동
1,2,청수 한양수자인 베스킨라빈스 앞,천안시 동남구 청당동 951,청당동,2021년,2024-05-29,127.152010,36.784222,청룡동
2,3,신방한라비발디 후문 정류장,천안시 동남구 신방동 1891,신방동,2021년,2024-05-29,127.134893,36.770837,신방동
3,4,신방우체국 중앙공원,천안시 동남구 신방동 1894,신방동,2021년,2024-05-29,127.133687,36.772472,신방동
4,5,통정 상쾌한 내과 앞,천안시 동남구 청당동 396-10,청당동,2021년,2024-05-29,127.142137,36.780822,청룡동
...,...,...,...,...,...,...,...,...,...
270,271,KG모빌리티 맞은편,천안시 서북구 쌍용동 440-2,쌍용동,2024년,2024-05-29,127.120426,36.791012,쌍용2동
271,272,충무로,천안시 서북구 쌍용동 440-25,쌍용동,2024년,2024-05-29,127.121019,36.791838,쌍용2동
272,273,49소갈비살/현대자동차 앞,천안시 서북구 쌍용동 450-2,쌍용동,2024년,2024-05-29,127.118048,36.790761,쌍용2동
273,274,광혜당 약국 앞 공중전화 부스 옆,천안시 서북구 쌍용동 824,쌍용동,2024년,2024-05-29,127.132424,36.807832,쌍용1동


In [19]:
my_df2.to_csv("./data/pm2.csv", index=False)

In [21]:
# 카카오 api로 되지 않은 것은 엑셀에서 직접 처리 -> pm3.csv

In [23]:
df = pd.read_csv("./data/pm3.csv")

In [27]:
df["법정동2"] = df["법정동2"].str.strip() 

In [29]:
df["법정동2"].unique()

array(['청룡동', '신방동', '불당2동', '불당1동', '백석동', '부성2동', '쌍용2동', '쌍용3동', '신안동',
       '봉명동', '부성1동', '성정2동', '쌍용1동', '성정1동', '성환읍', '문성동', nan, '일봉동',
       '중앙동', '원성1동', '원성2동'], dtype=object)

In [31]:
new_df = df.groupby("법정동2").size()

In [33]:
new_df = pd.DataFrame(new_df, columns=['개수'])

In [37]:
new_df = new_df.reset_index()

In [39]:
new_df["가중치"] = (new_df["개수"]-new_df["개수"].min())/(new_df["개수"].max()-new_df["개수"].min())

In [41]:
new_df = new_df.rename(columns={'법정동2': '주소'})
new_df = new_df.rename(columns={'가중치': 'pm가중치'})

In [45]:
new_df.to_csv("./data/pm_weight.csv", index=False)